In [2]:

import keras
from keras import backend as K
from keras.layers.core import Dense, Activation, Flatten
from keras.optimizers import adam
from keras.callbacks import ModelCheckpoint

from keras.metrics import categorical_crossentropy
from keras.preprocessing.image import ImageDataGenerator
from keras.preprocessing import image
from keras.models import Model
from keras.applications import imagenet_utils
from keras.layers import Dense,GlobalAveragePooling2D
from keras.applications import MobileNet
from keras.applications.mobilenet import preprocess_input
import numpy as np
from IPython.display import Image


Using TensorFlow backend.


In [4]:
train_path = 'C:/Users/Andy Cai/Desktop/Python/Hack Cambridge/Waste-classification-master/Waste-classification-master/waste-classification-data/DATASET/TRAIN'
test_path = 'C:/Users/Andy Cai/Desktop/Python/Hack Cambridge/Waste-classification-master/Waste-classification-master/waste-classification-data/DATASET/TEST'
#train_path = 'C:/Users/Andy Cai/Desktop/Python/Hack Cambridge/Waste-classification-master/Waste-classification-master/TRAIN'
#test_path = 'C:/Users/Andy Cai/Desktop/Python/Hack Cambridge/Waste-classification-master/Waste-classification-master/TEST'
IMG_BREDTH = 128
IMG_HEIGHT = 128
num_classes = 2

In [5]:
train_batch = ImageDataGenerator(featurewise_center=False,
                                 samplewise_center=False, 
                                 featurewise_std_normalization=False, 
                                 samplewise_std_normalization=False, 
                                 zca_whitening=False, 
                                 rotation_range=45, 
                                 width_shift_range=0.2, 
                                 height_shift_range=0.2, 
                                 horizontal_flip=True, 
                                 vertical_flip=False).flow_from_directory(train_path, 
                                                                          target_size=(IMG_HEIGHT, IMG_BREDTH), 
                                                                          classes=['O', 'R'], 
                                                                          batch_size=2)

test_batch = ImageDataGenerator().flow_from_directory(test_path, 
                                                      target_size=(IMG_HEIGHT, IMG_BREDTH), 
                                                      classes=['O', 'R'], 
                                                      batch_size=2)

Found 22564 images belonging to 2 classes.
Found 2513 images belonging to 2 classes.


In [6]:
base_model=MobileNet(weights='imagenet',include_top=False, input_shape=(128,128,3)) #imports the mobilenet model and discards the last 1000 neuron layer.

x=base_model.output
x=Flatten()(x)

preds=Dense(2,activation='softmax')(x) #final layer with softmax activation

model=Model(inputs=base_model.input,outputs=preds)



for layer in model.layers:
    layer.trainable=False
# or if we want to set the first 20 layers of the network to be non-trainable
for layer in model.layers[:20]:
    layer.trainable=False
for layer in model.layers[20:]:
    layer.trainable=True
    
    

def use_model(path):
    
    model = load_model('model4.h5')
    pic = plt.imread(path)
    pic = cv2.resize(pic, (IMG_BREDTH, IMG_HEIGHT))
    pic = np.expand_dims(pic, axis=0)
    classes = model.predict_classes(pic)
    
#     code using PIL
#     model = load_model('best_waste_classifier.h5')
#     pic1 = plt.imread(path)
#     pic = Image.open(path).resize((IMG_BREDTH, IMG_HEIGHT))
#     plt.imshow(pic1)
#     if model.predict_classes(np.expand_dims(pic, axis=0)) == 0:
#         classes = 'ORGANIC'
#     elif model.predict_classes(np.expand_dims(pic, axis=0)) == 1:
#         classes = 'RECYCLABLE'
    
    return classes

In [7]:
checkpoint = ModelCheckpoint('model4.h5', 
                             monitor='val_loss', 
                             verbose=0, 
                             save_best_only=True, 
                             mode='auto')

In [8]:
model.compile(loss='categorical_crossentropy', optimizer=adam(lr=1.0e-4), metrics=['accuracy'])

In [ ]:
# run code to train the neural network

model = model.fit_generator(train_batch,  
                             validation_data=test_batch,  
                             epochs=40, 
                             verbose=1, 
                             )

Epoch 1/40


In [61]:
print(use_model('3.jpg'))

[1]
